In [1]:
import os
import pandas as pd
import lyricsgenius
from dotenv import load_dotenv

# Cargar keys
load_dotenv()
GENIUS_API_KEY = os.getenv("GENIUS_API_KEY")

In [ ]:
#Inicializar Genius API
genius = lyricsgenius.Genius(GENIUS_API_KEY,timeout=15, retries=3)

filename = input("Introduce el nombre del archivo CSV a procesar (ej:): ../data/de_la_rose_top_tracks.csv): ")
artist_name = input("Introduce el nombre del artista: ")

df = pd.read_csv(filename)

columns = set(df.columns)
print(f"Columnas disponibles: {columns}")

possible_song_cols = [c for c in columns if 'song' in c.lower() or 'name' in c.lower()]

Columnas disponibles: {'release_date', 'popularity', 'song_name', 'album', 'preview_url'}


possible_song_cols = [c for c in columns if 'song' in c.lower() or 'name' in c.lower()]

1. columns
Es una lista de nombres de columnas (por ejemplo, de un DataFrame de Pandas):
columns = ['Track Name', 'Artist', 'Song Title', 'Duration_ms']

2. c for c in columns
Es un list comprehension, o comprensión de listas, que recorre cada elemento c en la lista columns.

3. c.lower()
Convierte cada nombre de columna a minúsculas para hacer la búsqueda insensible a mayúsculas:

'Track Name'.lower() → 'track name'
4. if 'song' in c.lower() or 'name' in c.lower()
Aplica un filtro: solo incluir columnas cuyo nombre contenga "song" o "name".

Va haciendo esto:

Iteración	c	c.lower()	¿Contiene 'song' o 'name'?	¿Se incluye en la lista final?
1	'Track Name'	'track name'	✅ Sí (name)	✅ Sí
2	'Artist'	'artist'	❌ No	❌ No
3	'Genre'	'genre'	❌ No	❌ No
4	'Song Title'	'song title'	✅ Sí (song)	✅ Sí

In [ ]:
song_col = possible_song_cols[0] if possible_song_cols else input("Columna de la canción")

print(f"Usando columna de canción: {song_col}")

def get_lyrics(row):
    try:
        song = genius.search_song(row[song_col], artist_name)
        return song.lyrics if song else ''
    except Exception as e:
        print(f"Error al obtener la letra de {row[song_col]}: {e}")
        return ''
    
df['lyrics'] = df.apply(get_lyrics, axis=1)

outfilename = filename.replace('.csv', '_letras.csv')
df.to_csv(outfilename, index=False)
print(f"Archivo con letras guardado: {outfilename}")

1. def get_lyrics(row):
Define una función que recibe una fila del DataFrame.

2. row[song_col]
Obtiene el nombre de la canción desde la fila.
song_col es la columna con los nombres de canciones (que detectaste antes).

3. genius.search_song(row[song_col], artist_name)
Usa la librería lyricsgenius para buscar la letra de la canción en Genius, combinando el nombre de la canción y el nombre del artista.

🧠 Requiere que artist_name sea definido antes como una cadena con el nombre del artista.

4. return song.lyrics if song else ''
Si se encuentra la canción (song no es None), devuelve su letra.
Si no, devuelve una cadena vacía ('').

5. except Exception as e:
Maneja errores (por ejemplo: si Genius no responde, hay caracteres raros, etc.).
Imprime el error y devuelve '' para esa fila.

6. df['lyrics'] = df.apply(get_lyrics, axis=1)
👉 Aplica la función get_lyrics() a cada fila del DataFrame.

axis=1 indica que trabaja fila por fila.

El resultado (las letras) se guarda en una nueva columna llamada 'lyrics'.

axis=0: aplica la función columna por columna (como "modo vertical")

axis=1: aplica la función fila por fila (como en tu caso)

🔍 El método .apply() se encarga de eso
Cuando haces:


df.apply(get_lyrics, axis=1)
Pandas automáticamente le pasa una fila a get_lyrics() en cada iteración.
Tú no ves ese row directamente aquí:

df.apply(get_lyrics, axis=1)
...pero en el fondo, es como si Pandas hiciera esto:

for fila in df.iterrows():
    letra = get_lyrics(fila)
📦 ¿Qué contiene row?
Es una especie de "mini-diccionario" con los datos de una sola fila. Por ejemplo, si tienes este DataFrame:

   song_name        artist
0  Shape of You     Ed Sheeran
1  Blinding Lights The Weeknd
Entonces:

En la primera vuelta, row['song_name'] → 'Shape of You'

En la segunda vuelta, row['song_name'] → 'Blinding Lights'

Y así para cada fila. Por eso la función get_lyrics(row) sabe qué canción buscar, gracias a esta línea dentro de ella:

python
Copiar
Editar
genius.search_song(row[song_col], artist_name)
Ahí:

song_col es el nombre de la columna con las canciones ('song_name')

row[song_col] accede al nombre de la canción de esa fila


----------------------------------------------------

✅ Entonces, sí entendiste bien:
row es la fila actual que apply() está procesando.

La función se ejecuta una vez por cada fila.

Tú decides cómo extraer los datos dentro de row usando row['nombre_columna'].